This file is a utility to download data from the Polybox into a local, specified data directory. The data is only downloaded if the data directory doesn't exist locally yet.

If you submit one main jupyter notebook that reproduces results, you might want to add these cells at the beginning of it.

In [1]:
def install_and_import(package):
    import importlib
    try:
        importlib.import_module(package)
    except ImportError:
        import pip
        pip.main(['install', package])
    finally:
        globals()[package] = importlib.import_module(package)

install_and_import('webdav4')

from webdav4.client import Client
import os
import getpass

def download_directory_webdav4(remote_path, local_path, client):
    for obj in client.ls(remote_path):
        p = obj['name']
        if client.isfile(obj['name']):
            # download file
            local_file_path = os.path.join(local_path, p.split("/")[-1])
            print(f"Downloading {p} to {local_file_path}")
            client.download_file(p, local_file_path)
        elif client.isdir(p):
            # download directory recursively
            local_dir_path = os.path.join(local_path, p.split("/")[-1])
            os.makedirs(local_dir_path, exist_ok=True)
            download_directory_webdav4(p, local_dir_path, client)

def get_data(
    local_data_dir,
    group_name,
):
    """
    Downloads data from ETH Polybox to a local directory.
    If the local_data_dir already exists, its contents will not be downloaded again.
    """
    if os.path.exists(local_data_dir):
        print(f"Directory {local_data_dir} already exists.")
        print("If you want to re-download the data, please delete it.")
        return
    
    eth_usr = input("you eth username: ")
    eth_pwd = getpass.getpass("your eth password: ")
    polybox_path=f'/Shared/CC2_GRACE_Submission/{group_name}/'
    url = f"https://{eth_usr}@polybox.ethz.ch/remote.php/dav/files/{eth_usr}/"
    
    client = Client(url, auth=(eth_usr, eth_pwd))
    
    assert client.exists(polybox_path), f"Destination {polybox_path} does not exist"

    # print(client.ls(polybox_path))
    os.makedirs(local_data_dir, exist_ok=True)
    download_directory_webdav4(polybox_path, local_data_dir, client)

Now, if you have not done so yet, edit the argument in below function call, then run the cell.

You will be prompted to type in your eth credentials to access the Polybox.

In [ ]:
get_data(
    local_data_dir='../Data',
    group_name='Group C',
)